In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install librosa
#!pip install tensorflow
import librosa
import librosa.display
import soundfile as sf
import random
import configparser
import math
#import tensorflow as tf
#import utility_functions as uf
import IPython.display as ipd
from IPython.display import Audio
from sklearn.model_selection import train_test_split
!pip install audiomentations
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
file_path = '/content/drive/MyDrive/extracted_features_with_labels.csv'
df = pd.read_csv(file_path)
df.head()


In [ ]:
df.shape

In [ ]:
df['Emotion'].value_counts()

In [ ]:

# Convert emotion labels to numerical values
label_encoder = LabelEncoder()
df['emotion_encoded'] = label_encoder.fit_transform(df['Emotion'])

Y = df['emotion_encoded'].values
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#encoder = OneHotEncoder()
#Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()


In [ ]:
Y.shape

In [ ]:
X = df.drop(['emotion_encoded','Emotion'],axis=1)
X.shape

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
# Print the shapes of the resulting sets
print("Training set features shape:", X_train.shape)
print("Testing set features shape:", X_test.shape)
print("Training set labels shape:", y_train.shape)
print("Testing set labels shape:", y_test.shape)

In [ ]:

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Assuming your data is in a pandas DataFrame `df`
# Separate features and labels
X = df.iloc[:, :-1]  # Features (all columns except 'emotion')
y = df['Emotion']    # Emotion labels (last column)


In [ ]:
X.drop('Emotion', axis=1, inplace=True)

In [ ]:

# Encode the emotion labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# List of unique emotions
emotions = label_encoder.classes_

In [ ]:
y_encoded

In [ ]:

## One-vs-All Classification
for emotion in emotions:
  # Create binary labels for one-vs-all classification (1 for the current emotion, 0 for others)
  y_bin = np.where(y_encoded == label_encoder.transform([emotion])[0], 1, 0)

    #Split data
  X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.3, random_state=42)

    # Train binary classifier
  clf = SVC(kernel='linear', probability=True)
  clf.fit(X_train, y_train)

    # Predict and evaluate
  y_pred = clf.predict(X_test)
  print(f"Results for emotion: {emotion}")
  print(classification_report(y_test, y_pred))

Results for emotion: angry
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1613
           1       0.81      0.56      0.66       231

    accuracy                           0.93      1844
   macro avg       0.88      0.77      0.81      1844
weighted avg       0.92      0.93      0.92      1844

Results for emotion: calm
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1621
           1       0.70      0.53      0.60       223

    accuracy                           0.92      1844
   macro avg       0.82      0.75      0.78      1844
weighted avg       0.91      0.92      0.91      1844



In [ ]:
!pip install shap
import shap
    Apply SHAP (use only if you have a model and data available)
#SHAP requires a fitted model and the data used to train the model
explainer = shap.Explainer(clf, X_train)
shap_values = explainer(X_test)

    # Plot SHAP values for the current emotion
shap.summary_plot(shap_values, X_test, feature_names=X.columns)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


# Filter for 5 emotions: Angry, Neutral, Calm, fearful, Surprised
emotions_to_keep = ['angry', 'neutral', 'calm', 'fearful', 'surprised']
filtered_data = df[df['emotion'].isin(emotions_to_keep)]

# Map emotions to integers if not already
emotion_map = {emotion: idx for idx, emotion in enumerate(emotions_to_keep)}
filtered_data['emotion'] = filtered_data['emotion'].map(emotion_map)

# Split features and labels
X = filtered_data.drop(columns=['emotion','emotion_encoded'])
y = filtered_data['emotion']


In [ ]:
X.shape

In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
y.unique()

In [ ]:

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import learning_curve

# Define base models
base_models = [
    ('svm', SVC(kernel='linear', probability=True)),
    ('rf', RandomForestClassifier()),
    ('logreg', LogisticRegression())
]

# Define meta-model
meta_model = LogisticRegression()

# Create the stacking ensemble
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Train the stacking ensemble model
stacking_clf.fit(X_train_scaled, y_train)

# Predict on test data
y_test_pred = stacking_clf.predict(X_test_scaled)
y_train_pred = stacking_clf.predict(X_train_scaled)

# Calculate accuracy for training and testing sets
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Plotting the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Classification Report
report = classification_report(y_test, y_test_pred, output_dict=True)

# Bar plot for precision, recall, and F1-score
metrics_df = pd.DataFrame(report).transpose()
metrics_df.drop(['support'], axis=1, inplace=True)

plt.figure(figsize=(10, 6))
metrics_df.iloc[:-1].plot(kind='bar', ax=plt.gca())
plt.title('Classification Report Metrics')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(title='Metrics')
plt.show()

# Plotting Learning Curves
train_sizes, train_scores, test_scores = learning_curve(
    stacking_clf, X_train_scaled, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10)
)

# Calculate the mean and std for train and test scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Plotting Learning Curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score')

# Plot the std deviation as shaded area
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, color='r', alpha=0.1)
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, color='g', alpha=0.1)

plt.title('Learning Curves')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.legend(loc='best')
plt.grid()
plt.show()


In [ ]:
print(classification_report(y_test, y_test_pred))

In [ ]:
import joblib

# Save the trained model to a file
joblib.dump(stacking_clf, 'Support_Vector_Machine_model.pkl')


In [ ]:
# Load your saved SVM model
model = joblib.load('Support_Vector_Machine_model.pkl')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import librosa
import os

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize KNN with k=5 (you can tune this)
knn = KNeighborsClassifier(n_neighbors=5)

# Train KNN model
knn.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = knn.predict(X_train_scaled)
y_test_pred = knn.predict(X_test_scaled)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix for KNN')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

In [ ]:
!pip install shap
import shap
    # Apply SHAP (use only if you have a model and data available)
#SHAP requires a fitted model and the data used to train the model
explainer = shap.Explainer(stacking_clf, X_train)
shap_values = explainer(X_test)

    # Plot SHAP values for the current emotion
shap.summary_plot(shap_values, X_test, feature_names=X.columns)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PCA for dimensionality reduction to 2D
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Scatter plot of the PCA-reduced training data
plt.figure(figsize=(8, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='viridis', s=50, alpha=0.7)
plt.colorbar()
plt.title("2D PCA Visualization of Speech Emotion Data (Training Set)")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.show()

In [ ]:
# Optional: Use t-SNE for a more complex visualization
tsne = TSNE(n_components=2)
X_train_tsne = tsne.fit_transform(X_train_scaled)

# Scatter plot of t-SNE reduced data
plt.figure(figsize=(8, 6))
plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap='coolwarm', s=50, alpha=0.7)
plt.colorbar()
plt.title("2D t-SNE Visualization of Speech Emotion Data (Training Set)")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Sample data
report = classification_report(y_test, y_test_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Heatmap visualization
plt.figure(figsize=(10, 6))
sns.heatmap(df_report.iloc[:-1, :-1], annot=True, cmap='Blues', fmt=".2f")
plt.title('Classification Report Heatmap')
plt.xlabel('Metrics')
plt.ylabel('Classes')
plt.show()


In [ ]:
!pip install shap
import shap
    # Apply SHAP (use only if you have a model and data available)
SHAP requires a fitted model and the data used to train the model
explainer = shap.Explainer(, X_train)
shap_values = explainer(X_test)

    # Plot SHAP values for the current emotion
shap.summary_plot(shap_values, X_test, feature_names=X.columns)


In [ ]:

plt.figure(figsize=(10, 8))
scatter = sns.scatterplot(x=X_train_tsne[:, 0], y=X_train_tsne[:, 1], hue=y_train, palette='Spectral', alpha=0.7)
plt.title('2D t-SNE Visualization of Speech Emotion Data')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')

# Create a colorbar based on the scatter plot
norm = plt.Normalize(vmin=y_train.min(), vmax=y_train.max())
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm)
cbar.set_label('Emotions')

plt.show()
